### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Cystic_Fibrosis/GSE60690'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Gene Expression Data Availability
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
def find_row_for_keyword(keyword, characteristics_dict):
    for key, values in characteristics_dict.items():
        unique_values = set(values)  # Ensure values have enough variability
        if any(keyword in value for value in values) and len(unique_values) > 1:
            return key, values
    return None, None

# Cystic_Fibrosis
trait_row, _ = find_row_for_keyword('persistent ps aer status', clinical_data)

# Age
age_row, _ = find_row_for_keyword('age of enrollment', clinical_data)

# Gender
gender_row, _ = find_row_for_keyword('Sex', clinical_data)

# 2.3 Data Type Conversion

# Function to extract the part after the colon
def extract_value(value):
    parts = value.split(':')
    result = parts[1].strip() if len(parts) > 1 else None
    return result

# Conversion for Cystic_Fibrosis (binary)
def convert_trait(value):
    extracted_value = extract_value(value)
    try:
        if extracted_value is None or extracted_value == 'NA':
            return None
        return int(float(extracted_value))  # converting through float to handle decimal numbers
    except (ValueError, TypeError):
        return None

# Conversion for Age (continuous)
def convert_age(value):
    extracted_value = extract_value(value)
    try:
        return float(extracted_value) if extracted_value is not None else None
    except (ValueError, TypeError):
        return None

# Conversion for Gender (binary: female -> 0, male -> 1)
def convert_gender(value):
    extracted_value = extract_value(value)
    try:
        if extracted_value is None:
            return None
        return 0 if extracted_value.lower() == 'female' else 1 if extracted_value.lower() == 'male' else None
    except (AttributeError, TypeError):
        return None

# Ensure the conversion functions are properly assigned
convert_trait = convert_trait
convert_age = convert_age
convert_gender = convert_gender

# 3. Save Metadata
save_cohort_info('GSE60690', './preprocessed/Cystic_Fibrosis/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Cystic_Fibrosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Cystic_Fibrosis/trait_data/GSE60690.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identifying the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'gene_assignment'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, gene_mapping)
